In [ ]:
import scipy.io
import numpy as np
from sklearn.metrics import auc
import pingouin as pg
import pandas as pd
from scipy.stats import f_oneway
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
from statistics import variance, mean


In [ ]:

hc_data_path = '/Users/jk1/unige_onedrive/OneDrive - unige.ch/BCT/atlas_BNA/BNA_240_flipped_N32_retroicor_SBB4_prop_bin_window/HC/CharPath240_binwin_HC.mat'
st_data_path = '/Users/jk1/unige_onedrive/OneDrive - unige.ch/BCT/atlas_BNA/BNA_240_flipped_N32_retroicor_SBB4_prop_bin_window/ST/CharPath240_binwin.mat'

In [ ]:
outcome_var = 'LocEfficiency'
minimum_connectivity_threshold = 0.3

In [ ]:
hc_data_mat = scipy.io.loadmat(hc_data_path)
st_data_mat = scipy.io.loadmat(st_data_path)

In [ ]:
n_thresholds = len(hc_data_mat[outcome_var][0][0])

In [ ]:
# converting matlab mat to arrays
# transposing to have shape (n_subj, n_thresholds)
hc_loc_eff = np.swapaxes(np.array(
                    [np.vstack(hc_data_mat[outcome_var][0][0][i]).T
                     for i in range(n_thresholds)]
                ), 0, -1)
st0_loc_eff = np.swapaxes(np.array(
                    [np.vstack(st_data_mat[outcome_var][0][0][0][0][0][i]).T
                     for i in range(n_thresholds)]
                ), 0, -1)
st1_loc_eff = np.swapaxes(np.array(
                    [np.vstack(st_data_mat[outcome_var][0][0][1][0][0][i]).T
                     for i in range(n_thresholds)]
                ), 0, -1)
st2_loc_eff = np.swapaxes(np.array(
                    [np.vstack(st_data_mat[outcome_var][0][0][2][0][0][i]).T
                     for i in range(n_thresholds)]
                ), 0, -1)

In [ ]:
n_hc = hc_loc_eff.shape[0]
n_st = st0_loc_eff.shape[0]

In [ ]:
# only analyse thresholds above minimum_connectivity_threshold
minimum_connectivity_threshold_index = int(minimum_connectivity_threshold*10 - 1)
connectivity_thresholds = np.arange(minimum_connectivity_threshold, 1.1, 0.1)

In [ ]:
# use auc only over predefined area of thresholds
def custom_auc(values_over_thresholds):
    return auc(connectivity_thresholds,
                values_over_thresholds[minimum_connectivity_threshold_index:])

In [ ]:
hc_loc_eff_auc = np.apply_along_axis(custom_auc, arr=hc_loc_eff, axis=-1)
st0_loc_eff_auc = np.apply_along_axis(custom_auc, arr=st0_loc_eff, axis=-1)
st1_loc_eff_auc = np.apply_along_axis(custom_auc, arr=st1_loc_eff, axis=-1)
st2_loc_eff_auc = np.apply_along_axis(custom_auc, arr=st2_loc_eff, axis=-1)

In [ ]:
all_subj_idx = np.arange(n_hc + n_st)
hc_idx = all_subj_idx[:n_hc]
st_idx = all_subj_idx[n_hc:]

In [ ]:
hc_loc_eff_auc_df = pd.DataFrame(data=hc_loc_eff_auc)
hc_loc_eff_auc_df['subject'] = hc_idx
hc_loc_eff_auc_df = hc_loc_eff_auc_df.melt(id_vars=['subject'], var_name='roi', value_name='loc_eff_auc')
hc_loc_eff_auc_df['timepoint'] = 0
hc_loc_eff_auc_df['group'] = 'hc'

In [ ]:
st0_loc_eff_auc_df = pd.DataFrame(data=st0_loc_eff_auc)
st0_loc_eff_auc_df['subject'] = st_idx
st0_loc_eff_auc_df = st0_loc_eff_auc_df.melt(id_vars=['subject'], var_name='roi', value_name='loc_eff_auc')
st0_loc_eff_auc_df['timepoint'] = 0
st0_loc_eff_auc_df['group'] = 'st'

In [ ]:
st1_loc_eff_auc_df = pd.DataFrame(data=st1_loc_eff_auc)
st1_loc_eff_auc_df['subject'] = st_idx
st1_loc_eff_auc_df = st1_loc_eff_auc_df.melt(id_vars=['subject'], var_name='roi', value_name='loc_eff_auc')
st1_loc_eff_auc_df['timepoint'] = 1
st1_loc_eff_auc_df['group'] = 'st'

In [ ]:
st2_loc_eff_auc_df = pd.DataFrame(data=st2_loc_eff_auc)
st2_loc_eff_auc_df['subject'] = st_idx
st2_loc_eff_auc_df = st2_loc_eff_auc_df.melt(id_vars=['subject'], var_name='roi', value_name='loc_eff_auc')
st2_loc_eff_auc_df['timepoint'] = 2
st2_loc_eff_auc_df['group'] = 'st'

In [ ]:
loc_eff_auc_df = pd.concat([hc_loc_eff_auc_df,st0_loc_eff_auc_df, st1_loc_eff_auc_df, st2_loc_eff_auc_df],
                            ignore_index=True)
all_st_loc_eff_auc_df = pd.concat([st0_loc_eff_auc_df, st1_loc_eff_auc_df, st2_loc_eff_auc_df],
                            ignore_index=True)

In [ ]:
# convert auc to numeric
loc_eff_auc_df['loc_eff_auc'] = loc_eff_auc_df['loc_eff_auc'].astype('float')
all_st_loc_eff_auc_df['loc_eff_auc'] = all_st_loc_eff_auc_df['loc_eff_auc'].astype('float')

# convert timepoint to numeric
loc_eff_auc_df['timepoint'] = loc_eff_auc_df['timepoint'].astype('float')
all_st_loc_eff_auc_df['timepoint'] = all_st_loc_eff_auc_df['timepoint'].astype('float')

In [ ]:
loc_eff_auc_df.head()

In [ ]:
sns.boxplot(x="timepoint", y="loc_eff_auc", hue="group", data=loc_eff_auc_df, palette="Set3")
plt.show()


In [ ]:
loc_eff_auc_df.to_csv('loc_eff_auc_df.csv', index=False)

## Post hoc analysis
Patient time points vs controls



In [ ]:
controls = loc_eff_auc_df[loc_eff_auc_df['group']=='hc']['loc_eff_auc']
print('Variance controls:' + str(variance(controls)))
mean(controls)

In [ ]:
# controls vs patients at all timepoints
ttest_ind(controls, st_tp0, equal_var=False)


In [ ]:
# controls vs patients at time point 0
st_tp0 = loc_eff_auc_df[(loc_eff_auc_df['group']=='st') & (loc_eff_auc_df['timepoint']==0)]['loc_eff_auc']
print('Variance subjects:' + str(variance(st_tp0)))
mean(st_tp0)

In [ ]:
ttest_ind(controls, loc_eff_auc_df[loc_eff_auc_df['group']=='st']['loc_eff_auc'], equal_var=False)

In [ ]:
# controls vs patients at time point 0
st_tp1 = loc_eff_auc_df[(loc_eff_auc_df['group']=='st') & (loc_eff_auc_df['timepoint']==1)]['loc_eff_auc']
print('Variance subjects:' + str(variance(st_tp1)))
mean(st_tp1)

In [ ]:
ttest_ind(controls, st_tp1, equal_var=False)